In [15]:
%%writefile plug_ai.slurm
#!/bin/bash
#SBATCH --job-name=plug_ai
#SBATCH --output=plug_ai.out
#SBATCH --error=plug_ai.out
#SBATCH --gres=gpu:1
#SBATCH --ntasks-per-node=1
#SBATCH --nodes=1
#SBATCH --hint=nomultithread
#SBATCH --time=00:50:00
#SBATCH --qos=qos_gpu-dev
#SBATCH --cpus-per-task=10
#SBATCH --account=sos@v100
#SBATCH -C v100-16g


## load Pytorch module
module purge

module load pytorch-gpu/py3/1.10.1

## launch script on every node
set -x

# code execution
srun python main.py --dataset_dir /gpfsscratch/idris/sos/ssos022/Medical/Task01_BrainTumour/ \
--limit_sample 20 --batch_size 2 --lr 0.0001

Overwriting plug_ai.slurm


In [16]:
%%bash
# submit job
sbatch plug_ai.slurm

Submitted batch job 937005


In [17]:
from threading import Event
import signal
from IPython.display import clear_output

def quit(signo, _frame):
    print("Interrupted by %d, shutting down" % signo)
    exit.set()

for sig in ('TERM', 'HUP', 'INT'):
    signal.signal(getattr(signal, 'SIG'+sig), quit);

exit = Event()

sq = !squeue -u $USER -n plug_ai
tail = !tail plug_ai.out
while len(sq) >= 2 and not exit.is_set():
    
    clear_output(wait=True)
    print(sq[0], sq[1], sep='\n')
    print(*tail, sep='\n')
    
    sq = !squeue -u $USER -n plug_ai
    tail = !tail plug_ai.out

    exit.wait(1)

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            937005   gpu_p13  plug_ai  ssos022  R       0:23      1 r10i6n1
[Epoch 1/1 |  Step_Epoch 1/10 | Loss 2.319648504257202]
[Epoch 1/1 |  Step_Epoch 2/10 | Loss 2.357532024383545]
[Epoch 1/1 |  Step_Epoch 3/10 | Loss 2.3537919521331787]
[Epoch 1/1 |  Step_Epoch 4/10 | Loss 2.3132243156433105]
[Epoch 1/1 |  Step_Epoch 5/10 | Loss 2.4168190956115723]
[Epoch 1/1 |  Step_Epoch 6/10 | Loss 2.3093342781066895]
[Epoch 1/1 |  Step_Epoch 7/10 | Loss 2.429335594177246]
[Epoch 1/1 |  Step_Epoch 8/10 | Loss 2.314669132232666]
[Epoch 1/1 |  Step_Epoch 9/10 | Loss 2.389770984649658]
[Epoch 1/1 |  Step_Epoch 10/10 | Loss 2.3313074111938477]


In [4]:
!scancel -u $USER -n plug_ai